In [50]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

In [51]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Seamless%20Translation/Track%20A/ukr_a_translated.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/test_eng.csv"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


File '/kaggle/working/ukr_a_translated.csv' already there; not retrieving.



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


File '/kaggle/working/test_eng.csv' already there; not retrieving.



In [52]:
df = pd.read_csv('/kaggle/working/ukr_a_translated.csv')
df

,id,text,anger,disgust,fear,joy,sadness,surprise
0,ukr_train_track_a_00001,I want a guitar and record covers.,0,0,0,0,0,0
1,ukr_train_track_a_00002,I'm not sure if I'm going to be able to get my...,0,0,0,1,0,0
2,ukr_train_track_a_00003,I'd like a little snow for the holidays for th...,0,0,0,0,0,0
3,ukr_train_track_a_00004,"When I first met you, you had a long, long hai...",0,0,0,0,0,0
4,ukr_train_track_a_00005,I'm so glad I didn't expect such a reaction.,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...
2461,ukr_train_track_a_02462,I'm in my third day of the marathon.,0,0,0,0,0,0
2462,ukr_train_track_a_02463,Dionysius could not refuse her request and wen...,0,0,0,0,0,0
2463,ukr_train_track_a_02464,The Russian hair was photographed by a woman i...,0,0,0,0,0,0
2464,ukr_train_track_a_02465,Evidence of a Broken Train in the Corn,0,0,0,0,0,0


In [53]:
df_test = pd.read_csv('/kaggle/input/remaining/ukr_translated.csv')
df_test

,id,text,anger,disgust,fear,joy,sadness,surprise
0,ukr_test_track_a_00001,"I'm going to Bunyuel. I'm alone, but I'm happy...",NaN,NaN,NaN,NaN,NaN,NaN
1,ukr_test_track_a_00002,"Wonderful, everyone is alive and well, celebra...",NaN,NaN,NaN,NaN,NaN,NaN
2,ukr_test_track_a_00003,"On the other hand, if I drew it, it would be m...",NaN,NaN,NaN,NaN,NaN,NaN
3,ukr_test_track_a_00004,"I want to take the girl to Samoriddh, but ther...",NaN,NaN,NaN,NaN,NaN,NaN
4,ukr_test_track_a_00005,I'm afraid of his words and his predictions.,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2229,ukr_test_track_a_02230,"""Who's eating in the 3rd car?"" crowd: ""Nobody!""",NaN,NaN,NaN,NaN,NaN,NaN
2230,ukr_test_track_a_02231,Can I die without suffering?,NaN,NaN,NaN,NaN,NaN,NaN
2231,ukr_test_track_a_02232,"And for now, sweet dreams.",NaN,NaN,NaN,NaN,NaN,NaN
2232,ukr_test_track_a_02233,Today I accidentally read an article about Por...,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Apply the clean_text function
df_test['text'] = df_test['text'].apply(clean_text)

# Step 3-5: (Same as your original code)
df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# Ensure emotion columns are integers
# emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness']
df_test[emotion_columns] = df_test[emotion_columns]

# Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_test_english.csv'
df_test.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.
Total samples: 2234

Emotion distribution in the dataset:
anger       0.0
disgust     0.0
fear        0.0
joy         0.0
sadness     0.0
surprise    0.0
dtype: float64

First few processed examples:
                       id                                               text  \
0  ukr_test_track_a_00001  i'm going to bunyuel. i'm alone, but i'm happy...   
1  ukr_test_track_a_00002  wonderful, everyone is alive and well, celebra...   
2  ukr_test_track_a_00003  on the other hand, if i drew it, it would be m...   
3  ukr_test_track_a_00004  i want to take the girl to samoriddh, but ther...   
4  ukr_test_track_a_00005       i'm afraid of his words and his predictions.   

   anger  disgust  fear  joy  sadness  surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    N

/tmp/ipykernel_30/3384993339.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [55]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
#emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# emotion_columns = ['anger', 'disgust', 'fear', 'joy', 'sadness']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_train_english.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.
Total samples: 2466

Emotion distribution in the dataset:
anger        98
disgust      86
fear        172
joy         412
sadness     333
surprise    196
dtype: int64

First few processed examples:
                                                text  anger  disgust  fear  \
0                 I want a guitar and record covers.      0        0     0   
1  I'm not sure if I'm going to be able to get my...      0        0     0   
2  I'd like a little snow for the holidays for th...      0        0     0   
3  When I first met you, you had a long, long hai...      0        0     0   
4       I'm so glad I didn't expect such a reaction.      0        0     0   

   joy  sadness  surprise  
0    0        0         0  
1    1        0         0  
2    0        0         0  
3    0        0         0  
4    1        0         1  


In [56]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6, problem_type="multi_label_classification")
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6, problem_type="multi_label_classification")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model for sequence classification
tokenizer_deberta = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model_deberta = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=6)  # Set num_labels according to your task

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
tokenizer_xlm_roberta = AutoTokenizer.from_pretrained("xlm-roberta-base")
model_xlm_roberta = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=6)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
from datasets import load_dataset, Dataset, DatasetDict

In [60]:
train_dataset = Dataset.from_pandas(df)

train_dataset

dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'],
        num_rows: 1972
    })
    test: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'],
        num_rows: 494
    })
})


In [61]:
# Convert train_dataset and val_dataset back to Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Save to CSV
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)

print("CSV files have been saved.")


CSV files have been saved.


In [62]:
train_df.head()

,text,anger,disgust,fear,joy,sadness,surprise
0,"Old man, all my life I've been afraid to look ...",0,0,1,0,0,0
1,It's like a cold wind blows through me.,0,0,0,0,1,0
2,I clarify that there were norms in that family...,0,0,0,0,0,0
3,I haven't dreamed of such horrors in a long ti...,0,0,1,0,0,0
4,"If you're looking for a #VivaLIFE, don't miss ...",0,0,0,0,0,0


In [63]:
val_df.head()

,text,anger,disgust,fear,joy,sadness,surprise
0,And I had almost lost hope and had time to cry...,0,0,0,0,1,0
1,"The Twitter-based social media platform, which...",0,0,0,0,0,0
2,"""Lunar"" is a euphemism for drug addiction.",0,0,0,0,0,0
3,You're still smoking on the move in this weather.,0,0,0,0,0,1
4,Who said anything about you?,0,0,0,0,0,1


In [64]:
cols = train_df.columns[1:]
cols

Index(['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'], dtype='object')

In [65]:
training_labels = train_df[cols]
training_labels = training_labels.to_numpy()
training_labels = training_labels.astype(np.float32)

train_sentences = train_df.text.values.tolist()

validation_labels = val_df[cols]
validation_labels = validation_labels.to_numpy()
validation_labels = validation_labels.astype(np.float32)

validation_sentences = val_df.text.values.tolist()

In [66]:
train_encodings = tokenizer(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(validation_sentences, padding=True, truncation=True, max_length=512)

In [67]:
from datasets import Dataset
import torch
import numpy as np

train_dataset_new = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': training_labels
})

validation_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': validation_labels
})

In [68]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

epochs = 2
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(df) * epochs)

training_arguments = TrainingArguments(
    output_dir="distilbert_base_model",
    num_train_epochs=8,  # Adjust the number of epochs as needed
    per_device_train_batch_size=32,  # Adjust based on your GPU memory
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="no",  # Do not save after each epoch
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    report_to=[],  # Disable W&B logging
    remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [69]:
#defining the metrics to be calculated during training
def multi_label_metrics(predictions, labels,):
    sigmoid = torch.nn.Sigmoid()
    predictions = sigmoid(torch.tensor(predictions))
    y_predictions = np.zeros(predictions.shape)
    y_true = labels
    y_predictions[np.where(predictions >= 0.5)] = 1
    accuracy = accuracy_score(y_true, y_predictions)
    f1 = f1_score(y_true, y_predictions, average='micro')
    auc = roc_auc_score(y_true, y_predictions, average='micro')

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc
    }

    return metrics

#defining the function to compute the metrics for each batch of data
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(preds, p.label_ids)

    return result

In [70]:
import wandb

# Initialize wandb with mode disabled
wandb.init(mode="disabled")

In [71]:
train_encodings_roberta = tokenizer_xlm_roberta(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings_roberta = tokenizer_xlm_roberta(validation_sentences, padding=True, truncation=True, max_length=512)

train_dataset_roberta = Dataset.from_dict({
    'input_ids': train_encodings_roberta['input_ids'],
    'attention_mask': train_encodings_roberta['attention_mask'],
    'labels': training_labels
})

validation_dataset_roberta = Dataset.from_dict({
    'input_ids': val_encodings_roberta['input_ids'],
    'attention_mask': val_encodings_roberta['attention_mask'],
    'labels': validation_labels
})


optimizer_roberta = AdamW(model_xlm_roberta.parameters(), lr=5e-5, eps=1e-8)
training_arguments_roberta = TrainingArguments(
    output_dir="roberta_base_model",
    num_train_epochs=8,  # Adjust the number of epochs as needed
    per_device_train_batch_size=32,  # Adjust based on your GPU memory
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="no",  # Do not save after each epoch
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    report_to=[],  # Disable W&B logging
    remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
)

trainer_roberta = Trainer(
    model=model_xlm_roberta,
    args=training_arguments_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=validation_dataset_roberta,
    compute_metrics=compute_metrics
)

trainer_roberta.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc
1,0.679800,0.619239,0.251012,0.098361,0.490106
2,0.527600,0.451213,0.522267,0.000000,0.500000
3,0.356300,0.284660,0.522267,0.000000,0.500000
4,0.275900,0.271567,0.522267,0.000000,0.500000
5,0.279900,0.266212,0.522267,0.000000,0.500000
6,0.275500,0.250194,0.522267,0.000000,0.500000
7,0.244100,0.244030,0.556680,0.213333,0.560834
8,0.237900,0.237270,0.576923,0.298780,0.593083


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=248, training_loss=0.37833818024204624, metrics={'train_runtime': 450.6441, 'train_samples_per_second': 35.008, 'train_steps_per_second': 0.55, 'total_flos': 2091709343225088.0, 'train_loss': 0.37833818024204624, 'epoch': 8.0})

In [72]:
model_xlm_roberta.save_pretrained("/kaggle/working/roberta_model")

In [73]:
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")

Cache emptied for GPU 0
Cache emptied for GPU 1


In [74]:
del model_xlm_roberta

In [75]:
del train_encodings_roberta
del val_encodings_roberta

In [76]:
# # Tokenize training and validation data
# train_encodings_deberta = tokenizer_deberta(train_sentences, padding=True, truncation=True, max_length=512)
# val_encodings_deberta = tokenizer_deberta(validation_sentences, padding=True, truncation=True, max_length=512)

# # Create datasets
# train_dataset_deberta = Dataset.from_dict({
#     'input_ids': train_encodings_deberta['input_ids'],
#     'attention_mask': train_encodings_deberta['attention_mask'],
#     'labels': training_labels
# })

# validation_dataset_deberta = Dataset.from_dict({
#     'input_ids': val_encodings_deberta['input_ids'],
#     'attention_mask': val_encodings_deberta['attention_mask'],
#     'labels': validation_labels
# })

# # Define optimizer, scheduler, and training arguments
# optimizer_deberta = AdamW(model_deberta.parameters(), lr=5e-5, eps=1e-8)
# training_arguments_deberta = TrainingArguments(
#     output_dir="deberta-base-model",
#     num_train_epochs=4,  # Adjust the number of epochs as needed
#     per_device_train_batch_size=32,  # Adjust based on your GPU memory
#     per_device_eval_batch_size=32,
#     evaluation_strategy="epoch",
#     save_strategy="no",  # Do not save after each epoch
#     logging_dir='./logs',
#     logging_steps=10,
#     warmup_steps=500,
#     weight_decay=0.01,
#     report_to=[],  # Disable W&B logging
#     remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
# )

# # Define the trainer and train
# trainer_deberta = Trainer(
#     model=model_deberta,
#     args=training_arguments_deberta,
#     train_dataset=train_dataset_deberta,
#     eval_dataset=validation_dataset_deberta,
#     compute_metrics=compute_metrics
# )

# trainer_deberta.train()

In [77]:
# model_deberta.save_pretrained("/kaggle/working/deberta_model")

In [78]:
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")

Cache emptied for GPU 0
Cache emptied for GPU 1


In [79]:
# del model_deberta

In [80]:
# del train_encodings_deberta
# del val_encodings_deberta

In [81]:
#defining the trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset_new,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc
1,0.659900,0.650212,0.214575,0.054146,0.472059
2,0.579100,0.507461,0.522267,0.000000,0.500000
3,0.370300,0.329373,0.522267,0.000000,0.500000
4,0.291800,0.284831,0.522267,0.000000,0.500000
5,0.284900,0.273158,0.522267,0.000000,0.500000
6,0.281400,0.264642,0.522267,0.000000,0.500000
7,0.257500,0.247013,0.558704,0.180556,0.549842
8,0.232600,0.222291,0.595142,0.313480,0.596940


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=248, training_loss=0.38421200888772167, metrics={'train_runtime': 260.9995, 'train_samples_per_second': 60.445, 'train_steps_per_second': 0.95, 'total_flos': 1387860580200960.0, 'train_loss': 0.38421200888772167, 'epoch': 8.0})

In [82]:
model.save_pretrained("/kaggle/working/distilbert_model")

In [83]:
del model

In [84]:
del train_encodings
del val_encodings

In [85]:
del trainer

In [86]:
# del trainer_deberta

In [87]:
del trainer_roberta

In [88]:
from transformers import DistilBertForSequenceClassification, DebertaV2ForSequenceClassification, XLMRobertaForSequenceClassification

# Load BERT model
model_bert = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/distilbert_model/")

# Load DeBERTa model
# model_deberta = DebertaV2ForSequenceClassification.from_pretrained("/kaggle/working/deberta_model/")

model_xlm_roberta = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/roberta_model/")

# Set models to eval mode
model_bert.eval()
# model_deberta.eval()
model_xlm_roberta.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [89]:
# Test data (assuming you have a DataFrame with a column 'text')
test_sentences = df_test['text'].tolist()

# Tokenize the test data
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings_deberta = tokenizer_deberta(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings_roberta = tokenizer_xlm_roberta(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [90]:
from torch.utils.data import DataLoader, TensorDataset

# Define a batch size
batch_size = 32  # Adjust based on your GPU memory

# Create DataLoader for test data
test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    # test_encodings_deberta['input_ids'],
    # test_encodings_deberta['attention_mask'],
    test_encodings_roberta['input_ids'],
    test_encodings_roberta['attention_mask']
)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Move models to the same device
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model_bert.to(device)
# model_deberta.to(device)
model_xlm_roberta.to(device)

# Initialize empty lists to store predictions
all_probabilities_bert = []
# all_probabilities_deberta = []
all_probabilities_roberta = []

# Process test data in batches
sigmoid = torch.nn.Sigmoid()

for batch in test_loader:
    # Unpack the batch
    input_ids_bert, attention_mask_bert, input_ids_roberta, attention_mask_roberta = [
        tensor.to(device) for tensor in batch
    ]

    # BERT Predictions
    with torch.no_grad():
        outputs_bert = model_bert(input_ids_bert, attention_mask=attention_mask_bert)
        probabilities_bert = sigmoid(outputs_bert.logits)
        all_probabilities_bert.append(probabilities_bert.cpu())

    # DeBERTa Predictions
    # with torch.no_grad():
    #     outputs_deberta = model_deberta(input_ids_deberta, attention_mask=attention_mask_deberta)
    #     probabilities_deberta = sigmoid(outputs_deberta.logits)
    #     all_probabilities_deberta.append(probabilities_deberta.cpu())

    # RoBERTa Predictions
    with torch.no_grad():
        outputs_roberta = model_xlm_roberta(input_ids_roberta, attention_mask=attention_mask_roberta)
        probabilities_roberta = sigmoid(outputs_roberta.logits)
        all_probabilities_roberta.append(probabilities_roberta.cpu())

    # Clear GPU memory
    for device_id in range(torch.cuda.device_count()):
        torch.cuda.set_device(device_id)
        torch.cuda.empty_cache()

# Concatenate all batch predictions
all_probabilities_bert = torch.cat(all_probabilities_bert, dim=0)
# all_probabilities_deberta = torch.cat(all_probabilities_deberta, dim=0)
all_probabilities_roberta = torch.cat(all_probabilities_roberta, dim=0)

# Models can now be deleted if further processing is not needed
del model_bert, model_xlm_roberta

# Final clearing of GPU memory
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()

# Output the predictions as numpy arrays
probabilities_bert_np = all_probabilities_bert.numpy()
# probabilities_deberta_np = all_probabilities_deberta.numpy()
probabilities_roberta_np = all_probabilities_roberta.numpy()

print("Inference completed successfully.")


Inference completed successfully.


In [91]:
# Collect the probabilities in a list
model_probabilities = [all_probabilities_bert, all_probabilities_roberta]

# Function to compute averaged probabilities
def average_probabilities(probabilities_list, selected_models):
    """
    Averages probabilities from selected models.

    Args:
    - probabilities_list: List of tensors containing probabilities from models.
    - selected_models: List of indices of models to include in the average (e.g., [0, 1] for BERT and DeBERTa).

    Returns:
    - Averaged probabilities tensor.
    """
    selected_probs = [probabilities_list[i] for i in selected_models]
    return torch.stack(selected_probs).mean(dim=0)

# Example Usage:
# Average all three models (BERT, DeBERTa, RoBERTa)
averaged_probabilities = average_probabilities(model_probabilities, selected_models=[0, 1])

# Alternative averages (uncomment as needed):
# averaged_probabilities = average_probabilities(model_probabilities, selected_models=[0, 1])  # BERT + DeBERTa
# averaged_probabilities = average_probabilities(model_probabilities, selected_models=[0, 2])  # BERT + RoBERTa
# averaged_probabilities = average_probabilities(model_probabilities, selected_models=[1, 2])  # DeBERTa + RoBERTa
# averaged_probabilities = model_probabilities[2]  # Use RoBERTa only

# Convert to binary predictions with a threshold
threshold = 0.4  # Adjust as needed
final_predictions = (averaged_probabilities > threshold).int()

# Print the shape and a few samples of the predictions
print("Final predictions shape:", final_predictions.shape)
print("Sample predictions:", final_predictions[:5])


Final predictions shape: torch.Size([2234, 6])
Sample predictions: tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]], dtype=torch.int32)


In [92]:
# Convert to a DataFrame
label_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# label_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness']
predicted_df_ensemble = pd.DataFrame(final_predictions.cpu().numpy(), columns=label_columns)

# Combine with the original test sentences

predicted_df_ensemble = predicted_df_ensemble.round().astype(int)

result_df_ensemble = pd.concat([df_test['text'], predicted_df_ensemble], axis=1)

# Display the results
print(result_df_ensemble.head())


                                                text  Anger  Disgust  Fear  \
0  i'm going to bunyuel. i'm alone, but i'm happy...      0        0     0   
1  wonderful, everyone is alive and well, celebra...      0        0     0   
2  on the other hand, if i drew it, it would be m...      0        0     0   
3  i want to take the girl to samoriddh, but ther...      0        0     1   
4       i'm afraid of his words and his predictions.      0        0     1   

   Joy  Sadness  Surprise  
0    0        0         0  
1    1        0         0  
2    0        0         0  
3    0        0         0  
4    0        0         0  


In [93]:
result_df_ensemble['id'] = [f"ukr_test_track_a_{str(i+1).zfill(5)}" for i in range(len(result_df_ensemble))]

# Step 4: Drop the original 'text' column
result_df_ensemble = result_df_ensemble.drop(columns=['text'])

# Step 5: Reorder the columns so 'id' comes first, followed by the emotion labels
# result_df_ensemble = result_df_ensemble[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]
result_df_ensemble = result_df_ensemble[['id','Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']]
# result_df_ensemble = result_df_ensemble[['id', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness']]


# Step 6: Save the final DataFrame as 'final_submission.csv' for submission
result_df_ensemble.to_csv('final_submission.csv', index=False)

# Display the first few rows of the final DataFrame
print(result_df_ensemble.head())

print("Submission file 'final_submission.csv' has been created.")

                       id  Anger  Disgust  Fear  Joy  Sadness  Surprise
0  ukr_test_track_a_00001      0        0     0    0        0         0
1  ukr_test_track_a_00002      0        0     0    1        0         0
2  ukr_test_track_a_00003      0        0     0    0        0         0
3  ukr_test_track_a_00004      0        0     1    0        0         0
4  ukr_test_track_a_00005      0        0     1    0        0         0
Submission file 'final_submission.csv' has been created.
